In [1]:
import scipy
import random
import math
import portpy.photon as pp
import numpy as np
from cvxpy import *
from scipy import sparse
import sklearn.metrics
import cProfile
import time
import sys
import psutil
import scipy.sparse as sp
import matplotlib.pyplot as plt
import scipy.sparse as coo_matrix
import scipy.sparse as csr_matrix

In [2]:
    data_dir = r'../data'
    data = pp.DataExplorer(data_dir=data_dir)
    patient_id = 'Lung_Phantom_Patient_1'
    data.patient_id = patient_id

    ct = pp.CT(data)
    structs = pp.Structures(data)
    beams = pp.Beams(data)

    opt_params = data.load_config_opt_params(protocol_name='Lung_2Gy_30Fx')
    structs.create_opt_structures(opt_params)

    inf_matrix_sparse = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams)

    protocol_name = 'Lung_2Gy_30Fx'
    clinical_criteria = pp.ClinicalCriteria(data, protocol_name)

    plan_sparse = pp.Plan(ct, structs, beams, inf_matrix_sparse, clinical_criteria)

 
    opt = pp.Optimization(plan_sparse, opt_params=opt_params)
    opt.create_cvxpy_problem()

    sol_sparse = opt.solve(solver='MOSEK', verbose=True)

   
    dose_sparse_1d = plan_sparse.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_sparse.get_num_of_fractions())

    beams_full = pp.Beams(data, load_inf_matrix_full=True)
   
    inf_matrix_full = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams_full, is_full=True)
    plan_full = pp.Plan(ct, structs, beams, inf_matrix_full, clinical_criteria)
    
    dose_full_1d = plan_full.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_full.get_num_of_fractions())


    A_full = plan_full.inf_matrix.A
    A_sparse = plan_sparse.inf_matrix.A


creating rinds.. This step may take some time due to dilation
rinds created!!
Creating BEV..
Loading sparse influence matrix...
Done
Objective Start
Objective done
Constraints Start
Structure ESOPHAGUS not available!
Structure ESOPHAGUS not available!
Constraints done
                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Jan 28 01:05:44 PM: Your problem has 1946 variables, 14 constraints, and 0 parameters.
(CVXPY) Jan 28 01:05:44 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 28 01:05:44 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 28 01:05:44 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 28 01:05:44 PM: Your problem is compiled with the CPP canonicalization backend.
----------------------------------------

(CVXPY) Jan 28 01:05:48 PM:   Primal.  obj: 4.2045390107e+01    nrm: 1e+02    Viol.  con: 1e-06    var: 5e-07    cones: 0e+00  
(CVXPY) Jan 28 01:05:48 PM:   Dual.    obj: 4.2045390142e+01    nrm: 6e+03    Viol.  con: 0e+00    var: 9e-09    cones: 0e+00  
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Jan 28 01:05:48 PM: Problem status: optimal
(CVXPY) Jan 28 01:05:48 PM: Optimal value: 4.205e+01
(CVXPY) Jan 28 01:05:48 PM: Compilation took 1.487e+00 seconds
(CVXPY) Jan 28 01:05:48 PM: Solver (including time spent in interface) took 1.822e+00 seconds
Creating BEV..
Loading full influence matrix..
Done


In [11]:

# تبدیل ماتریس متراکم به ماتریس پراکنده (CSR format)
sparse_matrix = sp.csr_matrix(A_full)


In [12]:
start_time = time.time()

norm_fro_squared =scipy.sparse.linalg.norm(A_full, ord='fro', axis=None)**2
norm_1 = np.abs(A_full).sum()
X = A_full.nonzero()[0]
Y = A_full.nonzero()[1]
squared_columns = sparse_matrix.multiply(A_full).sum(0)
squared_rows = sparse_matrix.multiply(A_full).sum(1)

data_omega = []
row_omega = []
column_omega = []
data_w = []

m,r,T = 1000,300,20  
n,d = A_full.shape

for i in range(len(X)):
    q = (m * ((squared_columns.item(0, Y[i]) + squared_rows.item(X[i], 0))/(2 * (n + d) * norm_fro_squared) + np.abs(A_full[X[i],Y[i]]) / norm_1))


    q_hat = q
    if 1 < q:
        q_hat = 1
    if q_hat > 0 :
        data_w.append(math.sqrt(1/q_hat))
    else:
        data_w.append(0)
    if random.random() < q_hat:
        data_omega.append(A_full[X[i], Y[i]])
        row_omega.append(X[i])
        column_omega.append(Y[i])

data_omega = np.array(data_omega)
row_omega = np.array(row_omega)
column_omega = np.array(column_omega)
data_w = np.array(data_w)
W = scipy.sparse.csr_matrix((data_w, (X, Y)), shape=(n, d))
W


r = 200
def partition (list_in, n):
    random.shuffle(list_in)
    return [list_in[i::n] for i in range(n)]

partitions = partition(np.arange(len(row_omega)), 2 * T + 1)
R0 = scipy.sparse.csr_matrix((data_omega[partitions[0]], (row_omega[partitions[0]], column_omega[partitions[0]])), shape=(n, d))
[U, S, V] = scipy.sparse.linalg.svds(R0.multiply(W).multiply(W), k=r)

for t in range(1):
    V = Variable((r, d))
    cost = 0
    datas = data_w[partitions[2 * t + 1]]
    data_R = data_omega[partitions[2 * t + 1]]
    row_R = row_omega[partitions[2 * t + 1]]
    column_R = column_omega[partitions[2 * t + 1]]
    for i in range(len(data_R)):
        temp = 0
        for j in range(r):
            temp += U[row_R[i], j] * V[j, column_R[i]] * datas[i]
        cost += (temp - data_R[i] * datas[i])**2
    prob = Problem(Minimize(cost))
    prob.solve(solver='MOSEK')
    print(prob.value)
    V = scipy.sparse.csr_matrix(V.value)
    print(len(V.nonzero()[0]))

n,d = A_full.shape
m,r,T = 1000,300,20    

sparsed_matrix = U @ V
sparsed_matrix

end_time = time.time()

total_execution_time = end_time - start_time
print(f"زمان کل اجرا: {total_execution_time} ثانیه")

C:\Users\user01\AppData\Roaming\Python\Python311\site-packages\cvxpy\problems\problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


357348.2541442041
0
زمان کل اجرا: 256.7787721157074 ثانیه


In [22]:
S = sparsed_matrix.T
A = A_full
#nonzero_count = np.count_nonzero(A_full)
#print("تعداد مقادیر غیر صفر ماتریس کامل: ", nonzero_count)
num_non_zero = np.count_nonzero(S @ A)
print("تعداد عناصر غیر صفر در ماتریس SA :", num_non_zero)

تعداد عناصر غیر صفر در ماتریس SA : 0


In [23]:
np.linalg.matrix_rank(S@A)

0

In [24]:
S.shape

(542, 60183)

In [25]:
##  بررسی  پراکندگی ماتریس
## چرا پراکنده نشد؟؟

is_sparse = sparse.issparse(sparsed_matrix)
is_sparse

False

In [26]:

sparsed_matrix_T = sparsed_matrix.transpose()


In [29]:
start_time = time.time()

sol_ls_A_Sparse = scipy.sparse.linalg.lsqr(S @ A, S @ d)


end_time = time.time()

total_execution_time = end_time - start_time
print(f"زمان حل کمترین مربعات{total_execution_time} ثانیه")

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [28]:
# جواب بهینه
x_star = sol_ls_A_Sparse[0]
print(f'x_star :\t {x_star[0:4]}')
   
#  تعداد مراحل اجرای الگوریتم
number_algorithm_execution_steps = sol_ls_A_Sparse[2]
print(f'number_algorithm_execution_steps :\t {number_algorithm_execution_steps}')

#  نرم بردار سمت راست d
Rightـhandـsideـvectorـnormـd = sol_ls_A_Sparse[3]
print(f'Rightـhandـsideـvectorـnormـd :\t {Rightـhandـsideـvectorـnormـd}')

#شاخص برای ارزیابی خروجی الگوریتم کمترین مربعات است	هرچقدر این نرم کوچکتر باشد نشانه بهتر بودن جواب بهینه است


#نرم ضرب داخلی ماتریس در جواب
l2_res = sol_ls_A_Sparse[4]
print(f'L2 norm of residuals :\t {l2_res}')

#   مجموع مربعات خطای سیستم 
sum_of_square1 = sol_ls_A_Sparse[5]
print(f'sum_of_square1 :\t {sum_of_square1}')

#   مجموع مربعات خطای ماتریس معکوس
sum_of_square2 = sol_ls_A_Sparse[6]
print(f'sum_of_square2 :\t {sum_of_square2}')

# بردار صفر              ?????????????
Zeroـvector = sol_ls_A_Sparse[9]
print(f'Zeroـvector :\t{Zeroـvector[0:24]}')

# کد بازگشتی 
Returnـcode = sol_ls_A_Sparse[1]
print(f'Returnـcode :\t{Returnـcode}')

NameError: name 'sol_ls_A_Sparse' is not defined

In [29]:

memory_usage_sparsed_matrix = sys.getsizeof(sparsed_matrix )
megabyte_size_sparsed_matrix= memory_usage_sparsed_matrix / 1024 / 1024

print(f"میزان حافظه ماتریس پراکنده : {megabyte_size_sparsed_matrix} مگابایت")

memory_usage_sol_ls_A_Sparse = sys.getsizeof(sol_ls_A_Sparse )
megabyte_size_sol_ls_A_Sparse = memory_usage_sol_ls_A_Sparse / 1024 / 1024

print(f"میزان حافظه کمترین مربعات : {megabyte_size_sol_ls_A_Sparse} مگابایت")

میزان حافظه ماتریس پراکنده : 0.0001220703125 مگابایت
میزان حافظه کمترین مربعات : 0.00011444091796875 مگابایت


<br><center> ***Minimize*** $\lVert Ax - d\rVert_{2}^2$
<br><center> ***subject to*** $x\succeq 0$

In [32]:
n,d = sparsed_matrix_T.shape

x = cp.Variable(n, nonneg=True)    

obj = cp.Minimize(cp.sum_squares(sparsed_matrix_T@A_full@x - sparsed_matrix_T@dose_full_1d))

prob = cp.Problem(obj)

In [33]:
start_time = time.time()

prob.solve()
objective_value = prob.value
print("Objective Value:", objective_value)

end_time = time.time()
total_execution_time = end_time - start_time
print(f"زمان کل اجرای کمترین مربعات مقید \t: {total_execution_time} ثانیه")


Objective Value: 0.0
زمان کل اجرای کمترین مربعات مقید 	: 1.718602180480957 ثانیه


In [34]:
memory_usage_objective_value = sys.getsizeof(objective_value )
megabyte_size_objective_value = memory_usage_objective_value / 1024 / 1024

print(f"میزان حافظه ی حل کمترین مربعات مقید با ماتریس کامل  :\t {megabyte_size_objective_value} مگابایت")

میزان حافظه ی حل کمترین مربعات مقید با ماتریس کامل  :	 2.288818359375e-05 مگابایت
